In [10]:
import numpy as np
import matplotlib.pyplot as plt
from TSpy.view import plot_mulvariate_time_series_and_label
from TSpy.label import reorder_label, seg_to_label
from TSpy.eval import *
from TSpy.utils import len_of_file
import scipy.io
import os

In [11]:
data_path = 'data/'
dataset = {'amc_86_01.4d':{'n_segs':4, 'label':{588:0,1200:1,2006:0,2530:2,3282:0,4048:3,4579:2}},
        'amc_86_02.4d':{'n_segs':8, 'label':{1009:0,1882:1,2677:2,3158:3,4688:4,5963:0,7327:5,8887:6,9632:7,10617:0}},
        'amc_86_03.4d':{'n_segs':7, 'label':{872:0, 1938:1, 2448:2, 3470:0, 4632:3, 5372:4, 6182:5, 7089:6, 8401:0}},
        'amc_86_07.4d':{'n_segs':6, 'label':{1060:0,1897:1,2564:2,3665:1,4405:2,5169:3,5804:4,6962:0,7806:5,8702:0}},
        'amc_86_08.4d':{'n_segs':9, 'label':{1062:0,1904:1,2661:2,3282:3,3963:4,4754:5,5673:6,6362:4,7144:7,8139:8,9206:0}},
        'amc_86_09.4d':{'n_segs':5, 'label':{921:0,1275:1,2139:2,2887:3,3667:4,4794:0}},
        'amc_86_10.4d':{'n_segs':4, 'label':{2003:0,3720:1,4981:0,5646:2,6641:3,7583:0}},
        'amc_86_11.4d':{'n_segs':4, 'label':{1231:0,1693:1,2332:2,2762:1,3386:3,4015:2,4665:1,5674:0}},
        'amc_86_14.4d':{'n_segs':3, 'label':{671:0,1913:1,2931:0,4134:2,5051:0,5628:1,6055:2}},
}

def dilate_label(label, f, max_len):
    slice_list = []
    for e in label:
        slice_list.append(e*np.ones(f, dtype=int))
    return np.concatenate(slice_list)[:max_len]

# Evaluate result on MoCap

In [12]:
def evaluation_on_MoCap():
    score_list = []
    for fname in dataset:
        # data = np.loadtxt('../data/MoCap/4d/'+fname)
        label = np.loadtxt('HVGHlearn/MoCap/'+fname+'/001/segm000.txt')[:,0].astype(int)
        groundtruth_json = dataset[fname]['label']
        groundtruth = seg_to_label(groundtruth_json)
        # print(groundtruth.shape)
        prediction = reorder_label(dilate_label(label, 100, len(groundtruth)))
        f_cut, p_cut, r_cut = evaluate_cut_point(groundtruth, prediction, 100)
        ari, anmi, nmi = evaluate_clustering(groundtruth, prediction)
        score_list.append(np.array([ari, anmi, nmi, f_cut, p_cut, r_cut]))
         # plot_mulvariate_time_series_and_label(data[0].T, label=prediction, groundtruth=groundtruth)
        print('ID: %s, ARI: %f, ANMI: %f, NMI: %f,  F1: %f, P: %f, R: %f' %(fname, ari, anmi, nmi, f_cut, p_cut, r_cut))
    score_list = np.vstack(score_list)
    print('AVG ---- ARI: %f, ANMI: %f, NMI: %f,  F1: %f, P: %f, R: %f' %(np.mean(score_list[:,0])\
        ,np.mean(score_list[:,1])
        ,np.mean(score_list[:,2])
        ,np.mean(score_list[:,3])
        ,np.mean(score_list[:,4])
        ,np.mean(score_list[:,5])))

# Evaluation on Synthetic

In [13]:
def evaluation_on_synthetic():
    score_list = []
    for i in range(100):
        groundtruth = np.loadtxt('../../data/synthetic_data_for_segmentation/test'+str(i)+'.csv', delimiter=',')[:,4].astype(int)    
        prediction = np.loadtxt('HVGHlearn/synthetic/test'+str(i)+'/001/segm000.txt')[:,0].astype(int)    
        prediction = dilate_label(prediction, 100, len(groundtruth))
        f1, precision, recall, ari, ami = evaluation(groundtruth, prediction)
        score_list.append(np.array([f1, precision, recall, ari, ami]))
        print('ID: %d, F1: %f, Precision: %f, Recall: %f,  ARI: %f, AMI: %f' %(i, f1, precision, recall, ari, ami))
    score_list = np.vstack(score_list)
    print('AVG ---- F1: %f, Precision: %f, Recall: %f,  ARI: %f, AMI: %f' %(np.mean(score_list[:,0])\
        ,np.mean(score_list[:,1])
        ,np.mean(score_list[:,2])
        ,np.mean(score_list[:,3])
        ,np.mean(score_list[:,4])))

# Evaluation on UCR-SEG

In [14]:
# HVGH refused to give result on some samples
# we skip thest samples.
def evaluation_on_UCR_SEG():
    score_list = []
    save_dir = 'HVGHlearn/UCR-SEG/'
    dataset_path = '../data/UCR-SEG/UCR_datasets_seg/'
    f_list = os.listdir(dataset_path)
    for fname in f_list:
        info_list = fname[:-4].split('_')
        f = info_list[0]
        seg_info = {}
        i = 0
        for seg in info_list[2:]:
            seg_info[int(seg)]=i
            i+=1
        seg_info[len_of_file(dataset_path+fname)]=i
        groundtruth = seg_to_label(seg_info)[:-1]
        if not os.path.exists(save_dir+fname):
            continue
        prediction = np.loadtxt('HVGHlearn/UCR-SEG/'+fname+'/001/segm000.txt')[:,0].astype(int)    
        prediction = dilate_label(prediction, 50, len(groundtruth))
        f1, precision, recall, ari, ami = evaluation(groundtruth, prediction)
        score_list.append(np.array([f1, precision, recall, ari, ami]))
        print('ID: %s, F1: %f, Precision: %f, Recall: %f,  ARI: %f, AMI: %f' %(f, f1, precision, recall, ari, ami))
    score_list = np.vstack(score_list)
    print('AVG ---- F1: %f, Precision: %f, Recall: %f,  ARI: %f, AMI: %f' %(np.mean(score_list[:,0])\
        ,np.mean(score_list[:,1])
        ,np.mean(score_list[:,2])
        ,np.mean(score_list[:,3])
        ,np.mean(score_list[:,4])))

# Evaluation on ActRecTut

In [15]:
def evaluation_on_ActRecTut():
    score_list = []
    dir_list = ['subject1_walk', 'subject2_walk']
    for dir_name in dir_list:
        dataset_path = os.path.join('../data/','ActRecTut/'+dir_name+'/data.mat')
        data = scipy.io.loadmat(dataset_path)
        groundtruth = data['labels'].flatten()
        prediction = np.loadtxt('HVGHlearn/ActRecTut/'+dir_name+'/001/segm000.txt')[:,0].astype(int)    
        prediction = dilate_label(prediction, 100, len(groundtruth))
        f1, precision, recall, ari, ami = evaluation(groundtruth, prediction)
        score_list.append(np.array([f1, precision, recall, ari, ami]))
        print('ID: %s, F1: %f, Precision: %f, Recall: %f,  ARI: %f, AMI: %f' %(dir_name, f1, precision, recall, ari, ami))
    score_list = np.vstack(score_list)
    print('AVG ---- F1: %f, Precision: %f, Recall: %f,  ARI: %f, AMI: %f' %(np.mean(score_list[:,0])\
        ,np.mean(score_list[:,1])
        ,np.mean(score_list[:,2])
        ,np.mean(score_list[:,3])
        ,np.mean(score_list[:,4])))

# Evaluation on PAMAP2

In [16]:
def evaluation_on_PAMAP2():
    score_list = []
    for i in range(1,10):
        groundtruth = np.loadtxt('../../data/PAMAP2/Protocol/subject10'+str(i)+'.dat')[:,1].astype(int)    
        prediction = np.loadtxt('HVGHlearn/PAMAP2/subject10'+str(i)+'/001/segm000.txt')[:,0].astype(int)    
        prediction = dilate_label(prediction, 100, len(groundtruth))
        f_cut, p_cut, r_cut = evaluate_cut_point(groundtruth, prediction, 100)
        ari, anmi, nmi = evaluate_clustering(groundtruth, prediction)
        score_list.append(np.array([ari, anmi, nmi, f_cut, p_cut, r_cut]))
         # plot_mulvariate_time_series_and_label(data[0].T, label=prediction, groundtruth=groundtruth)
        print('ID: %d, ARI: %f, ANMI: %f, NMI: %f,  F1: %f, P: %f, R: %f' %(i, ari, anmi, nmi, f_cut, p_cut, r_cut))
    score_list = np.vstack(score_list)
    print('AVG ---- ARI: %f, ANMI: %f, NMI: %f,  F1: %f, P: %f, R: %f' %(np.mean(score_list[:,0])\
        ,np.mean(score_list[:,1])
        ,np.mean(score_list[:,2])
        ,np.mean(score_list[:,3])
        ,np.mean(score_list[:,4])
        ,np.mean(score_list[:,5])))

In [17]:
def effect_of_length():
    time_list = np.loadtxt('HVGHlearn/effect_of_length/time9.txt')
    print(time_list)

def effect_of_dimension():
    time_list = np.loadtxt('HVGHlearn/effect_of_dim/time20.txt')
    print(time_list)

In [18]:
# evaluation_on_MoCap()
evaluation_on_synthetic()
# evaluation_on_UCR_SEG()
# evaluation_on_ActRecTut()
# evaluation_on_PAMAP2()
# effect_of_length()
# effect_of_dimension()



OSError: HVGHlearn/synthetic/test0/001/segm000.txt not found.